In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

from langchain_core.prompts import ChatPromptTemplate

In [ ]:
loader = PyPDFLoader("attention.pdf")
docs = loader.load()
docs

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

In [ ]:
documents=text_splitter.split_documents(docs)
documents

In [ ]:
data_base = FAISS.from_documents(documents[:30],OpenAIEmbeddings())
data_base

In [ ]:
query="An attention function can be described as mapping a query "
result = data_base.similarity_search(query)
result[0].page_content

In [ ]:
#load Ollama llama2 LLM model

llm=Ollama(model="llama2")
llm

In [ ]:
#design ChatPrompt Template


prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")


In [ ]:
#chain introduction
#create stuff document chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
retriever = data_base.as_retriever()
retriever

In [ ]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input":"Scaled Dot-Product Attention"})

In [ ]:
response['answer']